# Email Phishing Detection Model

## About Dataset

The Email Phishing Dataset, compiled by user Ethan Cratchley on kaggle, is a combination of two different datasets containing "safe" emails from The Enron Email Dataset, along with phishing and "safe" emails from The Phishing Email Dataset. The dataset includes features such as the number of words, unique words, stopwords, links, unique domains, email addresses, and spelling errors, along with associated labels. The purpose of this dataset is to use machine learning to train a model that would be able to detect phishing emails. Companies and corporations can use this model to identify phishing emails, preventing attacks, and protecting sensitive information and data. In addition, Cybersecurity teams can use the model to find patterns in phishing emails, allowing them to finetune and improve the model.

# 1. Data Preprocessing


- Perform exploratory data analysis (EDA) to gain insights into the dataset's structure and distributions.
- Handle missing values, outliers, and any inconsistencies in the data.
- Encode categorical variables and normalize numerical features as necessary.
- Split the dataset into training and testing sets, ensuring a proper balance of classes.
- Discuss your applied techniques to tackle data imbalancedness if you are working on an imbalanced dataset



In [1]:
# importing initial libraries needed
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(color_codes=True)

In [2]:
df = pd.read_csv('email_phishing_data.csv')
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'email_phishing_data.csv'